In [ ]:
import numpy as np
import pandas as pd

"Irrelevant" class - raw data & pre-processing

In [ ]:
#Reading the csv files containing tweets for "Irrelevant" class. 

ibb_anlamsiz = pd.read_csv('data/ibb_anlamsiz.csv', encoding = 'cp1254')
ibb_siyasi = pd.read_csv('data/ibb_siyasi.csv', encoding = 'cp1254')

In [ ]:
!pip install TurkishStemmer

In [ ]:
import nltk
from TurkishStemmer import TurkishStemmer
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
"""This is the preprocessing for tweets. We have additional processes such as removal of incomplete words at the end of tweets, usernames 
and RT (denoting retweets) to remove extra evidence that is not related to the content."""

porter = TurkishStemmer()

stop_words = nltk.corpus.stopwords.words('turkish')

def clean_text(string):

    message = re.sub(r'\w+\…'," ", string) #Removal of incomplete words
    message = re.sub(r'@\w+(:)?'," ", message) #Removal of user names
    message = re.sub(r'\ART', " ", message) #Removal of RT
    message = re.sub(r'\b[\w\-.]+?@\w+?\.\w{2,4}\b', 'emailaddr', message)
    message = re.sub(r'(http[s]?\S+)|(\w+\.[A-Za-z]{2,4}\S*)', ' ', #Replace URLs with space at this one because it might be too freq in this class
                     message)
    message = re.sub(r'₺|\$', 'money', message) #Replace money symbols with 'moneysymb'
    message = re.sub(
        r'\b(\+\d{1,2}\s)?\d?[\-(.]?\d{3}\)?[\s.-]?\d{3}[\s.-]?\d{4}\b', #Replace phone numbers with 'phonenumbr'
        'phonenumbr', message)
    message = re.sub(r'\d+(\.\d+)?', 'numbr', message)  #Replace numbers with 'numbr'
    message = re.sub(r'[^\w\d\s]', ' ', message) #Punctuation removed
    message = re.sub(r'\s+', ' ', message) #Too much space replaced by single space
    message = re.sub(r'^\s+|\s+?$', '', message.lower()) #Get rid of spaces at the beginning and end.
    return ' '.join(
    porter.stem(term)
    for term in message.split()
    if term not in set(stop_words)
    )

In [ ]:
"""Creating the dataset for tweets labelled as 'Irrelevant'. We apply pre-processing, arrange the columns
and add the label "4" to denote 'Irrelevant'."""

ibb_ilgisiz = pd.concat([ibb_anlamsiz, ibb_siyasi], axis = 0)

textCopy = ibb_ilgisiz['Text']
textCopy = textCopy.apply(clean_text)
ibb_ilgisiz["Text"] = textCopy
ibb_ilgisiz.reset_index(inplace = True)
ibb_ilgisiz.drop(['index', 'Author', 'Date'], axis = 1, inplace = True)
ibb_ilgisiz['Label'] = 4
ibb_ilgisiz.shape

(2713, 2)

In [ ]:
#Train-validation and test split for 'Irrelevant' dataset. 

from sklearn.model_selection import train_test_split

tr_irrel_text, tst_irrel_text, tr_irrel_lbl, tst_irrel_lbl = train_test_split(ibb_ilgisiz.drop('Label', axis = 1), ibb_ilgisiz['Label'], test_size = 0.2)

tr_irrel_text, val_irrel_text, tr_irrel_lbl, val_irrel_lbl = train_test_split(tr_irrel_text, tr_irrel_lbl, test_size = 0.2)

Municipality's enterprises data - raw data & pre-processing

In [ ]:
uploaded2 = files.upload()

Saving ibb_organisations_test.csv to ibb_organisations_test.csv
Saving ibb_organisations_train.csv to ibb_organisations_train.csv
Saving ibb_organisations_valid.csv to ibb_organisations_valid.csv


In [ ]:
#Reading the files that contain complaints to municipality's enterprises. We already had them as split into train-valid-test sets. 

tr_organisations = pd.read_csv('data/ibb_organisations_train.csv')
val_organisations = pd.read_csv('data/ibb_organisations_valid.csv')
tst_organisations = pd.read_csv('data/ibb_organisations_test.csv')

In [ ]:
#Preprocessing procedure for complaints to municipality's enterprises

porter = TurkishStemmer()

stop_words = nltk.corpus.stopwords.words('turkish')

def clean_text2(string):
    message = re.sub(r'\b[\w\-.]+?@\w+?\.\w{2,4}\b', 'emailaddr', str(string))
    message = re.sub(r'(http[s]?\S+)|(\w+\.[A-Za-z]{2,4}\S*)', 'httpaddr', #Replace URLs with 'httpaddr'
                     message)
    message = re.sub(r'₺|\$', 'money', message) #Replace money symbols with 'moneysymb'
    message = re.sub(
        r'\b(\+\d{1,2}\s)?\d?[\-(.]?\d{3}\)?[\s.-]?\d{3}[\s.-]?\d{4}\b', #Replace phone numbers with 'phonenumbr'
        'phonenumbr', message)
    message = re.sub(r'\d+(\.\d+)?', 'numbr', message)  #Replace numbers with 'numbr'
    message = re.sub(r'[^\w\d\s]', ' ', message)
    message = re.sub(r'\s+', ' ', message)
    message = re.sub(r'^\s+|\s+?$', '', message.lower())
    return ' '.join(
    porter.stem(term)
    for term in message.split()
    if term not in set(stop_words)
    )

In [ ]:
#Pre-processing implemented on train-valid-test data sets of municipality's enterprises. 

textCopy = tr_organisations['comment']
textCopy = textCopy.apply(clean_text2)
tr_organisations["comment"] = textCopy

textCopy1 = val_organisations['comment']
textCopy1 = textCopy1.apply(clean_text2)
val_organisations["comment"] = textCopy1

textCopy1 = tst_organisations['comment']
textCopy1 = textCopy1.apply(clean_text2)
tst_organisations["comment"] = textCopy1

In [ ]:
#Arranging column names.

tr_organisations.columns = ['Text', 'Label']
val_organisations.columns = ['Text', 'Label']
tst_organisations.columns = ['Text', 'Label']

Combining "Irrelevant" class with the enterprises

In [ ]:
#Combining labels of irrelevant class with that of enterprises 

tr_irrel_lbl = pd.DataFrame(tr_irrel_lbl, columns = ['Label'])
val_irrel_lbl = pd.DataFrame(val_irrel_lbl, columns = ['Label'])
tst_irrel_lbl = pd.DataFrame(tst_irrel_lbl, columns = ['Label'])

y_train = pd.concat([tr_organisations['Label'], tr_irrel_lbl['Label']], axis = 0, ignore_index = True)
y_valid = pd.concat([val_organisations['Label'], val_irrel_lbl['Label']], axis = 0, ignore_index = True)
y_test = pd.concat([tst_organisations['Label'], tst_irrel_lbl['Label']], axis = 0, ignore_index = True)

In [ ]:
#Combining texts of irrelevant class with that of enterprises

new_train = pd.concat([tr_organisations['Text'], tr_irrel_text['Text']], axis = 0, ignore_index = True)
new_valid = pd.concat([val_organisations['Text'], val_irrel_text['Text']], axis = 0, ignore_index = True)
new_test = pd.concat([tst_organisations['Text'], tst_irrel_text['Text']], axis = 0, ignore_index = True)

In [ ]:
#Adding up text and label columns + shuffling the datasets

import random

new_train_with_y = pd.concat([new_train, y_train], axis = 1)
new_valid_with_y = pd.concat([new_valid, y_valid], axis = 1)
new_test_with_y = pd.concat([new_test, y_test], axis = 1)

new_train_with_y = new_train_with_y.sample(frac = 1, random_state = 42).reset_index(drop = True)
new_valid_with_y = new_valid_with_y.sample(frac = 1, random_state = 42).reset_index(drop = True)
new_test_with_y = new_test_with_y.sample(frac = 1, random_state = 42).reset_index(drop = True)

In [ ]:
#Separating test and labels for train-valid-test each and updating new_train, new_valid, new_test

new_train = new_train_with_y['Text']
y_train = new_train_with_y['Label']

new_valid = new_valid_with_y['Text']
y_valid = new_valid_with_y['Label']

new_test = new_test_with_y['Text']
y_test = new_test_with_y['Label']

Tokenization

In [ ]:
# Tokenizing the sentences

count_vectorizer = CountVectorizer()
vocab_fit = count_vectorizer.fit_transform(new_train.values)
train_vec = pd.DataFrame(vocab_fit.toarray(), columns=count_vectorizer.get_feature_names(), index=new_train.index)

vocab_fit = count_vectorizer.transform(new_valid.values)
validation1_vec = pd.DataFrame(vocab_fit.toarray(), columns=count_vectorizer.get_feature_names(), index=new_valid.index)

vocab_fit = count_vectorizer.transform(new_test.values)
test_vec = pd.DataFrame(vocab_fit.toarray(), columns=count_vectorizer.get_feature_names(), index=new_test.index)

In [ ]:
#Removing exceptionally short sentences from the datasets

train_vec['sum'] = train_vec.apply(sum, axis = 1)
validation1_vec['sum'] = validation1_vec.apply(sum, axis = 1)
test_vec['sum'] = test_vec.apply(sum, axis = 1)

train_vec2 = train_vec.loc[train_vec['sum'] >= 5]
validation1_vec2 = validation1_vec.loc[validation1_vec['sum'] >= 5]
test_vec2 = test_vec.loc[test_vec['sum'] >= 5]

y_train2 = y_train.loc[train_vec['sum'] >= 5]
y_valid2 = y_valid.loc[validation1_vec['sum'] >= 5]
y_test2 = y_test.loc[test_vec['sum'] >= 5]

train_vec2.drop('sum', axis = 1, inplace = True)
validation1_vec2.drop('sum', axis = 1, inplace = True)
test_vec2.drop('sum', axis = 1, inplace = True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
#We will need these word_vectors later for find_label(). 

total_vec = pd.concat([train_vec2, validation1_vec2, test_vec2], axis = 0)
y_labels = pd.concat([y_train2, y_valid2, y_test2], axis = 0)

ML algorithms and their results

In [ ]:
#LogisticRegression results

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report

lr_model = LogisticRegression(dual = True, solver = 'liblinear', max_iter = 5000)
lr_model.fit(train_vec2, y_train2)
y_pred_lr = lr_model.predict(validation1_vec2)

print(classification_report(y_valid2, y_pred_lr))
print('\n')
print(confusion_matrix(y_valid2, y_pred_lr))
print('\n')
print('Accuracy:', accuracy_score(y_valid2, y_pred_lr))
print('\n')
print('f1 score:', f1_score(y_valid2, y_pred_lr, average = 'weighted'))

              precision    recall  f1-score   support

           0       0.94      0.96      0.95       532
           1       0.95      0.96      0.95       480
           2       0.94      0.90      0.92       441
           3       0.96      0.94      0.95       433
           4       0.97      1.00      0.98       411

    accuracy                           0.95      2297
   macro avg       0.95      0.95      0.95      2297
weighted avg       0.95      0.95      0.95      2297



[[511   0  15   1   5]
 [  0 460   4  12   4]
 [ 26  11 396   5   3]
 [  4  13   8 406   2]
 [  0   1   0   0 410]]


Accuracy: 0.9503700478885503


f1 score: 0.950135369480399


In [ ]:
#SGDClassifier Results 

from sklearn.linear_model import SGDClassifier

sgd_model = SGDClassifier()
sgd_model.fit(train_vec2, y_train2)
y_pred_sgd = sgd_model.predict(validation1_vec2)

print(classification_report(y_valid2, y_pred_sgd))
print('\n')
print(confusion_matrix(y_valid2, y_pred_sgd))
print('\n')
print('Accuracy:', accuracy_score(y_valid2, y_pred_sgd))
print('\n')
print('f1 score:', f1_score(y_valid2, y_pred_sgd, average = 'weighted'))

              precision    recall  f1-score   support

           0       0.94      0.95      0.95       532
           1       0.95      0.95      0.95       480
           2       0.93      0.89      0.91       441
           3       0.93      0.94      0.93       433
           4       0.97      1.00      0.98       411

    accuracy                           0.94      2297
   macro avg       0.94      0.94      0.94      2297
weighted avg       0.94      0.94      0.94      2297



[[504   1  17   6   4]
 [  0 456   6  13   5]
 [ 26  11 393  10   1]
 [  4  12   8 405   4]
 [  0   2   0   0 409]]


Accuracy: 0.9434044405746626


f1 score: 0.9431772152222709


In [ ]:
#Pipeline created for Random Forest

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, KFold, cross_val_score

pipe = Pipeline([('classifier', RandomForestClassifier())])

# Create param grid.

param_grid = [
    {'classifier' : [RandomForestClassifier()],
    'classifier__n_estimators' : list(range(10,101,10)),
    'classifier__max_features' : list(range(6,32,5))}
]

# Create grid search object

clf = GridSearchCV(pipe, param_grid = param_grid, cv = 5, verbose=True, n_jobs=-1)

# Fit on data

best_clf = clf.fit(train_vec2, y_train2)
y_pred_clf = clf.predict(validation1_vec2)


print(classification_report(y_valid2, y_pred_clf))
print('\n')
print(confusion_matrix(y_valid2, y_pred_clf))
print('\n')
print('Accuracy:', accuracy_score(y_valid2, y_pred_clf))
print('\n')
print('f1 score:', f1_score(y_valid2, y_pred_clf, average = 'weighted'))

Fitting 5 folds for each of 60 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed: 18.2min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 29.4min finished


              precision    recall  f1-score   support

           0       0.85      0.95      0.89       532
           1       0.92      0.97      0.94       480
           2       0.91      0.74      0.82       441
           3       0.94      0.86      0.90       433
           4       0.93      0.99      0.96       411

    accuracy                           0.90      2297
   macro avg       0.91      0.90      0.90      2297
weighted avg       0.91      0.90      0.90      2297



[[505   0  16   5   6]
 [  1 464   0  11   4]
 [ 81  18 328   7   7]
 [ 10  22  15 371  15]
 [  0   2   1   0 408]]


Accuracy: 0.9037875489769265


f1 score: 0.9018071258793048


In [ ]:
#Multinomial NB results

from sklearn.naive_bayes import MultinomialNB

mb_model = MultinomialNB()
mb_model.fit(train_vec2, y_train2)

y_pred_mb = mb_model.predict(validation1_vec2) 

print(classification_report(y_valid2, y_pred_mb))
print('\n')
print(confusion_matrix(y_valid2, y_pred_mb))
print('\n')
print('Accuracy:', accuracy_score(y_valid2, y_pred_mb))
print('\n')
print('f1 score:', f1_score(y_valid2, y_pred_mb, average = 'weighted'))

              precision    recall  f1-score   support

           0       0.86      0.95      0.90       532
           1       0.92      0.97      0.95       480
           2       0.91      0.78      0.84       441
           3       0.91      0.96      0.93       433
           4       1.00      0.90      0.95       411

    accuracy                           0.91      2297
   macro avg       0.92      0.91      0.91      2297
weighted avg       0.92      0.91      0.91      2297



[[505   1  20   6   0]
 [  1 466   0  13   0]
 [ 72  14 345  10   0]
 [  4   9   5 415   0]
 [  5  14   8  14 370]]


Accuracy: 0.9146713104048759


f1 score: 0.9138576118995212


Demonstration of find_label function in examples

In [ ]:
#Regular preprocessing for normal texts (we do not expect to have tweets) - now as a generator

porter = TurkishStemmer()

stop_words = nltk.corpus.stopwords.words('turkish')

def clean_text3(string):
    message = re.sub(r'\b[\w\-.]+?@\w+?\.\w{2,4}\b', 'emailaddr', str(string))
    message = re.sub(r'(http[s]?\S+)|(\w+\.[A-Za-z]{2,4}\S*)', 'httpaddr', #Replace URLs with 'httpaddr'
                     message)
    message = re.sub(r'₺|\$', 'money', message) #Replace money symbols with 'moneysymb'
    message = re.sub(
        r'\b(\+\d{1,2}\s)?\d?[\-(.]?\d{3}\)?[\s.-]?\d{3}[\s.-]?\d{4}\b', #Replace phone numbers with 'phonenumbr'
        'phonenumbr', message)
    message = re.sub(r'\d+(\.\d+)?', 'numbr', message)  #Replace numbers with 'numbr'
    message = re.sub(r'[^\w\d\s]', ' ', message)
    message = re.sub(r'\s+', ' ', message)
    message = re.sub(r'^\s+|\s+?$', '', message.lower())
    yield ' '.join(
    porter.stem(term)
    for term in message.split()
    if term not in set(stop_words)
    )

In [ ]:
#This function applies pre-processing on any string it receives, vectorizes it and tries to predict its class with MultinomialNB. 

def find_label(sentence):

  sentence = [sentence]

  new_sentence = clean_text3(sentence)

  new_vocab_vec = count_vectorizer.transform(new_sentence)
  
  new_pred = mb_model.predict(new_vocab_vec)

  if new_pred == 0:
    return ('İgdaş')
  elif new_pred == 1:
    return ('İett')
  elif new_pred == 2:
    return ('İski')
  elif new_pred == 3:
    return ('Diğer İBB iştirakleri')
  elif new_pred == 4:
    return ('İlgisiz')

In [ ]:
#The sentence below would be translated as "Rudeness of the bus driver" - which would concern İett. 

find_label('Otobüs şoförünün büyük ayıbı')

'İett'

In [ ]:
#The sentence below would be translated as "Exorbitant bills for gas" - which would concern İgdaş.

find_label('Fahiş doğalgaz faturası!')

'İgdaş'